<a href="https://colab.research.google.com/github/rafmacalaba/Credit-Scoring-Kaggle-Competition/blob/master/%5BNER_Finetuning%5D_GLiNER_Data_Use_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# use all synth norm only
# use manually annotated data only
# add more negatives on the manual annotated data

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!pip install gliner accelerate
!pip install --upgrade datasets huggingface_hub
!pip install transformers==4.52.4

In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [4]:
from gliner.multitask.base import GLiNERBasePipeline
from typing import Optional, List, Union
from datasets import load_dataset, Dataset
from gliner import GLiNER
from gliner.training import Trainer, TrainingArguments
from gliner.data_processing.collator import DataCollator

In [5]:
import torch

def load_model(model_path):
  # Device and model setup
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model = GLiNER.from_pretrained(model_path).to(device)

  return model

In [6]:
import json
def load_json_data(filepath: str):
    """
    Loads a JSON or JSONL file from the given path.
    Returns a dict or list (for JSON) or list of dicts (for JSONL).
    """
    with open(filepath, 'r', encoding='utf-8') as f:
        first_line = f.readline()
        f.seek(0)
        if first_line.strip().startswith('{') and not first_line.strip().endswith('['):
            # Try JSON Lines: each line is a dict
            try:
                data = [json.loads(line) for line in f if line.strip()]
                if len(data) == 1:
                    f.seek(0)
                    data = json.load(f)
                return data
            except Exception:
                f.seek(0)
                return json.load(f)
        else:
            return json.load(f)

In [7]:
# all_synthetic = load_json_data("/content/drive/MyDrive/colab-artifacts/refugee_data/REFUGEE_RE_POS/_prevent_overfitting/cleaned_train_ready/synthetic/all_synthetic_normalized.jsonl")
all_synthetic = load_json_data("/content/drive/MyDrive/colab-artifacts/refugee_data/REFUGEE_RE_POS/_prevent_overfitting/cleaned_train_ready/FINAL_PLEASE/synthetic-final/all_synthetic_normalized.jsonl")
all_syntheticv2 = load_json_data("/content/drive/MyDrive/colab-artifacts/refugee_data/REFUGEE_RE_POS/_prevent_overfitting/cleaned_train_ready/FINAL_PLEASE/synthetic-final/cleaned_synthetic_v0-4files.jsonl")

In [8]:
len(all_synthetic), len(all_syntheticv2)

(4137, 2625)

In [9]:
all_synthetic = all_synthetic + all_syntheticv2

In [10]:
def drop_duplicate_records(records):
    """
    Drop duplicate records based on their 'ner' content.
    Keeps the first occurrence and removes later duplicates.
    """
    seen = set()
    unique = []
    for rec in records:
        # Use frozenset of tuples so order inside ner doesn't matter
        ner_key = frozenset(tuple(item) for item in rec.get("ner", []))
        if ner_key not in seen:
            seen.add(ner_key)
            unique.append(rec)
    return unique

In [11]:
all_synthetic = drop_duplicate_records(all_synthetic)

In [12]:
len(all_synthetic)

4892

In [13]:
import glob

# all_train_fnames = glob.glob("/content/drive/MyDrive/colab-artifacts/refugee_data/REFUGEE_RE_POS/_prevent_overfitting/cleaned_train_ready/train/*.jsonl")

In [15]:
!ls /content/drive/MyDrive/colab-artifacts/refugee_data/REFUGEE_RE_POS/_prevent_overfitting/cleaned_train_ready/FINAL_PLEASE/train-final

all_retained_records_payload_done_reviewed.jsonl
reviewed_all_pdfs_train_ready.jsonl
structured
tr_holdout_train_ready.jsonl
tr_holdout_with_classification_train_ready.jsonl


In [ ]:
# all_train_fnames = ['/content/drive/MyDrive/colab-artifacts/refugee_data/REFUGEE_RE_POS/_prevent_overfitting/cleaned_train_ready/train/manual_labelled_trainready.jsonl',
#                    '/content/drive/MyDrive/colab-artifacts/refugee_data/REFUGEE_RE_POS/_prevent_overfitting/cleaned_train_ready/train/INITIAL_DATA.jsonl',
#                      '/content/drive/MyDrive/colab-artifacts/refugee_data/REFUGEE_RE_POS/_prevent_overfitting/cleaned_train_ready/train/CONVERTED_HOLDOUT.jsonl',
#                     '/content/drive/MyDrive/colab-artifacts/refugee_data/REFUGEE_RE_POS/_prevent_overfitting/cleaned_train_ready/train/reviewed_train_ready.jsonl'
#                      ]


In [16]:
all_train_fnames = glob.glob("/content/drive/MyDrive/colab-artifacts/refugee_data/REFUGEE_RE_POS/_prevent_overfitting/cleaned_train_ready/FINAL_PLEASE/train-final/*.jsonl")

In [17]:
len(all_train_fnames)

4

In [18]:
all_train_fnames

['/content/drive/MyDrive/colab-artifacts/refugee_data/REFUGEE_RE_POS/_prevent_overfitting/cleaned_train_ready/FINAL_PLEASE/train-final/all_retained_records_payload_done_reviewed.jsonl',
 '/content/drive/MyDrive/colab-artifacts/refugee_data/REFUGEE_RE_POS/_prevent_overfitting/cleaned_train_ready/FINAL_PLEASE/train-final/reviewed_all_pdfs_train_ready.jsonl',
 '/content/drive/MyDrive/colab-artifacts/refugee_data/REFUGEE_RE_POS/_prevent_overfitting/cleaned_train_ready/FINAL_PLEASE/train-final/tr_holdout_train_ready.jsonl',
 '/content/drive/MyDrive/colab-artifacts/refugee_data/REFUGEE_RE_POS/_prevent_overfitting/cleaned_train_ready/FINAL_PLEASE/train-final/tr_holdout_with_classification_train_ready.jsonl']

In [19]:
all_train = []
for train_fn in all_train_fnames:
    all_train.extend(load_json_data(train_fn))

In [20]:
def keep_only_dataset_spans(records):
    """
    Keep only NER spans that are datasets:
    'named dataset', 'unnamed dataset', or 'vague dataset',
    and exclude anything that contains '<>'.
    """
    allowed = {"named dataset", "unnamed dataset", "vague dataset"}
    cleaned_records = []
    for rec in records:
        rec_copy = rec.copy()
        rec_copy["ner"] = [
            span for span in rec.get("ner", [])
            if span[2] in allowed and "<>" not in span[2]
        ]
        cleaned_records.append(rec_copy)
    return cleaned_records

In [21]:
all_train = keep_only_dataset_spans(all_train)
all_synthetic = keep_only_dataset_spans(all_synthetic)

In [22]:
print(f'''Total train entries loaded: {len(all_train)} \n
Total synthetic entries loaded: {len(all_synthetic)}''')

Total train entries loaded: 2194 

Total synthetic entries loaded: 4892


In [23]:
len([c for c in all_train if c['ner']])

1483

In [24]:
len([c for c in all_train if not c['ner']])

711

In [25]:
# import os
# import random
# from typing import List, Dict, Any, Tuple

# def make_train_test_split(
#     token_level_data: List[Dict[str, Any]],
#     test_size: float = 0.1,
#     seed: int = 42
# ) -> Tuple[List[Dict[str, Any]], List[Dict[str, Any]]]:
#     """
#     Simple random train/test split.
#     """
#     random.seed(seed)

#     n = len(token_level_data)
#     n_test = max(1, int(test_size * n))

#     shuffled = token_level_data[:]
#     random.shuffle(shuffled)

#     test = shuffled[:n_test]
#     train = shuffled[n_test:]

#     os.makedirs("assets", exist_ok=True)
#     return train, test

import os
import random
from typing import List, Dict, Any, Tuple

def make_train_test_split(
    token_level_data: List[Dict[str, Any]],
    test_size: float = 0.1,
    seed: int = 42
) -> Tuple[List[Dict[str, Any]], List[Dict[str, Any]]]:
    """
    Train/test split with stratification on whether 'ner' is empty.
    Ensures both splits have examples with and without datasets.
    """
    random.seed(seed)

    # Separate positives and negatives
    positives = [rec for rec in token_level_data if rec.get("ner") and len(rec["ner"]) > 0]
    negatives = [rec for rec in token_level_data if not rec.get("ner") or len(rec["ner"]) == 0]

    def split_group(group: List[Dict[str, Any]]) -> Tuple[List[Dict[str, Any]], List[Dict[str, Any]]]:
        n_total = len(group)
        n_test = max(1, int(n_total * test_size)) if n_total > 0 else 0
        random.shuffle(group)
        return group[n_test:], group[:n_test]  # (train, test)

    # Split both groups separately
    train_pos, test_pos = split_group(positives)
    train_neg, test_neg = split_group(negatives)

    # Merge back
    train = train_pos + train_neg
    test = test_pos + test_neg

    # Shuffle final sets
    random.shuffle(train)
    random.shuffle(test)

    return train, test


In [26]:
def collect_unique_labels(*datasets):
    labels = set()
    for data in datasets:
        for rec in data:
            for span in rec.get("ner", []):
                if len(span) > 2:   # make sure it's well-formed
                    labels.add(span[2])
    return labels

unique_labels = collect_unique_labels(all_synthetic, all_train)
print("Unique labels:", unique_labels)

Unique labels: {'unnamed dataset', 'vague dataset', 'named dataset'}


In [27]:
def train_gliner_model(
    synthetic_dataset,
    validated_dataset,
):

  # prefinetune
  model = load_model('knowledgator/gliner-multitask-v1.0')
  # model = load_model("knowledgator/gliner-multitask-large-v0.5")
  synth_train, synth_test = make_train_test_split(synthetic_dataset)
  data_collator = DataCollator(model.config, data_processor=model.data_processor, prepare_labels=True)
  # calculate number of epochs
  num_steps = 2000
  batch_size = 16
  data_size = len(synth_train)
  num_batches = data_size // batch_size
  num_epochs = 2#max(1, num_steps // num_batches)

  training_args = TrainingArguments(
      output_dir="models",
      learning_rate=5e-5,
      weight_decay=0.01,
      others_lr=1e-5,
      others_weight_decay=0.01,
      lr_scheduler_type="cosine", #for prefinetuning
      warmup_ratio=0.1,
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size,
      focal_loss_alpha=0.75,
      focal_loss_gamma=2,
      num_train_epochs=num_epochs,
      eval_strategy="steps",
      save_steps = 500,
      save_total_limit=10,
      dataloader_num_workers = 0,
      use_cpu = False,
      report_to="none",
      )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=synth_train,
      eval_dataset=synth_test,
      data_collator=data_collator,
  )

  trainer.train()


  ### finetuning

  validated_train, validated_test = make_train_test_split(validated_dataset)
  data_collator = DataCollator(model.config, data_processor=model.data_processor, prepare_labels=True)

  # calculate number of epochs
  num_steps = 1000
  batch_size = 8
  data_size = len(validated_train)
  num_batches = data_size // batch_size
  num_epochs = 1#max(1, num_steps // num_batches)
  training_args = TrainingArguments(
      output_dir="models",
      learning_rate=5e-6,
      weight_decay=0.01,
      others_lr=1e-5,
      others_weight_decay=0.01,
      lr_scheduler_type="linear", #fine tuning
      warmup_ratio=0.1,
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size,
      focal_loss_alpha=0.75,
      focal_loss_gamma=2,
      num_train_epochs=num_epochs,
      eval_strategy="steps",
      save_steps = 500,
      save_total_limit=10,
      dataloader_num_workers = 0,
      use_cpu = False,
      report_to="none",
      )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=validated_train,
      eval_dataset=validated_test,
      data_collator=data_collator,
  )

  trainer.train()

  return trainer.model

In [28]:
torch.cuda.empty_cache()

In [ ]:
init_model = train_gliner_model(all_synthetic, all_train)

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

README.md: 0.00B [00:00, ?B/s]

gliner_config.json: 0.00B [00:00, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/970 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Step,Training Loss,Validation Loss


In [ ]:
init_model.push_to_hub("rafmacalaba/datause-extraction-v0", commit_message="all manual and reviewed with hard negatives")

In [ ]:
text = 'C O U N T R Y F O C U S R E P O R T 2 0 2 3 - T A N Z A N I A 27 15. United Republic of Tanzania. URT 2021. Report on Nationally Determined Contributions. Vice President ’ s Office, Dodoma. 16. United Republic of Tanzania. URT 2013. Report on Forests, Rangelands and Climate Change Adaptation in Tanzania. Vice President ’ s Office, Dar es Salaam. 17. United Republic of Tanzania, URT ( 2014-a ). State of the Environment Report, Vice President ’ s Office, Vice President ’ s Office, Dar es Salaam 18. United Republic of Tanzania, URT ( 2014-b ), Tanzania ’ s Second Communication to the United Nations Framework Convention on Climate Change, Vice President ’ s Office, Dar es Salaam 19. URT 2012. Tanzania Long Term Perspective Plan ( LTPP ) 2011 / 12 – 2025 / 26. The Roadmap to Middle Income Country. 20.'
# text = 'The 2020 Demographic and Health Survey (DHS), conducted by the Ministry of Health in Kenya, provides data on maternal health and child nutrition for rural households.'

In [ ]:
init_model.predict_entities(text, labels=['named dataset', 'unnamed dataset', 'vague dataset'])

In [ ]:
labels = ["named dataset"]

entities = init_model.predict_entities(text, labels)

for entity in entities:
    print(entity["text"], "=>", entity["label"])

In [ ]:
from google.colab import runtime

runtime.unassign()

In [ ]:
# init_model = GLiNER.from_pretrained("rafmacalaba/datause-extraction-v5")